<a href="https://colab.research.google.com/github/kovzanok/ml/blob/main/hw_language_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
# Скачиваем архив
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

# Распаковываем
!tar -xzf aclImdb_v1.tar.gz


--2025-07-18 14:08:57--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  59.7MB/s    in 1.3s    

2025-07-18 14:08:59 (59.7 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

from collections import Counter
from typing import List
from itertools import chain
import string

import seaborn
seaborn.set(palette='summer')

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [5]:
import os

def load_imdb_data(split='train'):
    data_path = f'aclImdb/{split}'
    texts, labels = [], []
    for label in ['pos', 'neg']:
        folder = os.path.join(data_path, label)
        for fname in os.listdir(folder):
            with open(os.path.join(folder, fname), encoding='utf-8') as f:
                texts.append(f.read())
                labels.append(1 if label == 'pos' else 0)
    return texts, labels

train_texts, train_labels = load_imdb_data('train')
test_texts, test_labels = load_imdb_data('test')

print(f"Пример: {train_labels[0]} → {train_texts[0][:200]}...")


Пример: 1 → "The College Girl Murders" is my first acquaintance with the writing work of Edgar Wallace  and generally my first real acquaintance with "Krimi" films in general  and I can say that I'm moderately ...


In [6]:
sentences = [ sent_tokenize(text,language='english') for text in tqdm(train_texts) ]
sentences = list(chain(*sentences))

word_threshold = 32

sentences = list(filter(lambda s:len(s.split())<=word_threshold, sentences))# Получить отдельные предложения и поместить их в sentences

  0%|          | 0/25000 [00:00<?, ?it/s]

In [7]:
print("Всего предложений:", len(sentences))
print("Всего предложений после фильтрации по кол-во слов:", len(sentences))

Всего предложений: 223314
Всего предложений после фильтрации по кол-во слов: 223314


Посчитаем для каждого слова его встречаемость.

In [8]:
words = Counter()
stop_words = set(stopwords.words('english'))

for sentence in tqdm(sentences):
    words_list = word_tokenize(sentence)
    filtered_words_list = [ word.lower() for word in words_list if ((word.lower() not in stop_words) and (word.isalpha()) and word.lower()!='br')]
    words.update(filtered_words_list)
# Расчет встречаемости слов

words.most_common(10)

  0%|          | 0/223314 [00:00<?, ?it/s]

[('movie', 29302),
 ('film', 23794),
 ('one', 15913),
 ('like', 12169),
 ('good', 9628),
 ('would', 8123),
 ('even', 7756),
 ('really', 7683),
 ('time', 7364),
 ('see', 7260)]

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [9]:
vocab_size = 40000
vocab = set([ word for word, _ in words.most_common(vocab_size)])
vocab.add('<unk>')
vocab.add('<bos>')
vocab.add('<eos>')
vocab.add('<pad>')
# Наполнение словаря

In [10]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [11]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 40004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [12]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [13]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        sentence = self.data[idx]
        words = word_tokenize(sentence)

        tokenized_sentence = [self.bos_id]
        for word in words:
           ind = word2ind.get(word, self.unk_id)
           tokenized_sentence.append(ind)
        tokenized_sentence.append(self.eos_id)
        # Допишите код здесь

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [14]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [15]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 64

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [21]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch['input_ids'], batch['target_ids']
            x, y = x.to(device), y.to(device)

            logits = model(x)

            logits = logits.view(-1, logits.size(-1))
            y = y.reshape(-1)

            loss = criterion(logits, y)
            total_loss += loss.item()
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity, total_loss / len(dataloader)

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [22]:
def train_epoch(model, criterion, optimizer, train_loader):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        x, y = batch['input_ids'], batch['target_ids']
        x, y = x.to(device), y.to(device)


        logits = model(x)

        logits = logits.view(-1, logits.size(-1))
        y = y.reshape(-1)

        loss = criterion(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

In [23]:
def train_model(model, criterion, optimizer, train_loader, eval_loader, epochs=10):
    for i in tqdm(range(epochs)):
        epoch_loss = train_epoch(model, criterion, optimizer, train_loader)
        epoch_perplexity, eval_loss = evaluate(model, criterion, eval_loader)
        print(f'Epoch {i+1}/{epochs}, Train Loss: {epoch_loss:.4f}, Eval Loss: {eval_loss:.4f}, Perplexity: {epoch_perplexity:.4f}')

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [24]:
class LanguageModel(nn.Module):
    def __init__(self,
                 embedding_dim,
                 vocab_len,
                 rnn_type = 'lstm',
                 num_layers = 1
                ):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_len, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, embedding_dim, num_layers, batch_first=True) if rnn_type == 'lstm' else nn.GRU(embedding_dim, embedding_dim, num_layers, batch_first=True)
        self.projection = nn.Linear(embedding_dim, vocab_len)

        self.non_linear = nn.Tanh()

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        embeddings = self.embeddings(input_batch)
        output, _ = self.rnn(embeddings)
        output = self.projection(self.non_linear(output))

        return output

In [ ]:
model = LanguageModel(128, len(vocab))
model = model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_model(model, criterion, optimizer, train_dataloader, eval_dataloader)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2792 [00:00<?, ?it/s]

  0%|          | 0/1745 [00:00<?, ?it/s]

Epoch 1/10, Train Loss: 3.3785, Eval Loss: 3.2282, Perplexity: 25.4183


  0%|          | 0/2792 [00:00<?, ?it/s]

...

  0%|          | 0/2792 [00:00<?, ?it/s]

  0%|          | 0/1745 [00:00<?, ?it/s]

Epoch 8/10, Train Loss: 2.8836, Eval Loss: 2.9079, Perplexity: 18.4290


  0%|          | 0/2792 [00:00<?, ?it/s]

  0%|          | 0/1745 [00:00<?, ?it/s]

### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [ ]:
# Проведите второй эксперимент

### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.